In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import sklearn
import seaborn as sns
from sklearn import metrics
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
%matplotlib inline
style.use("ggplot")

accuracy = metrics.accuracy_score

/Users/kerrylam/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("titanic_training.csv", header = 0) 

In [3]:
del train['Name'], train['Ticket'], train['Cabin'], train['PassengerId']

In [4]:
train = pd.get_dummies(train, columns = ['Sex', 'Embarked'], drop_first = True)
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,1


In [5]:
del train["Embarked_C"]
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,1


In [6]:
train.fillna(method = 'ffill', inplace = True) # Impute Missing Data
train = train.astype(float) # Convert DF Type to Float

"""Split Data"""
train = train[0:len(train)]
test = train[len(train):]

"""Normalize Data for Faster Computation"""
train = train/train.max().astype(np.float64)
test = test/test.max().astype(np.float64)

In [7]:
cols = train.columns.tolist()
cols.insert(0, cols.pop(cols.index('Survived')))
train = train.reindex(columns= cols)
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0.0,1.000000,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0
1,1.0,0.333333,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0
2,1.0,1.000000,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0
3,1.0,0.333333,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0
4,0.0,1.000000,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0


In [8]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 0


In [9]:
X = pd.DataFrame(train.ix[:,1:])
target = train["Survived"]
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.000000,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0
1,0.333333,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0
2,1.000000,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0
3,0.333333,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0
4,1.000000,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0


In [10]:
train_, test_, targ_train, targ_test = train_test_split(X, target, test_size = 0.40, random_state = None)

In [11]:
print ("Train:       ", train.shape)
print ("Train Class: ", targ_train.shape)
print ("Test:        ", test_.shape)
print ("Test Class:  ", targ_test.shape)

Train:        (891, 9)
Train Class:  (534,)
Test:         (357, 8)
Test Class:   (357,)


In [12]:
train_ = train.reset_index()
test_ = test_.reset_index()

In [13]:
del train_["index"], test_["index"]

### Part 1: 

Partition Training Data into 5 Folds:

- Create train_meta with same rows and IDs as training data
- Create Empty Columns: M1 and M2 (NaNs)

- Create test_meta variable with same rows and IDs as test dataset 
- Create Empty Columns: M1 and M2 (NaNs)

In [14]:
"""Create Train and Test Meta"""
train_meta = pd.DataFrame.copy(train_)
train_meta["M1"] = np.nan
train_meta["M2"] = np.nan
print(train.shape)
train_meta.head()

(891, 9)


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,0.0,1.000000,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,NaN,NaN
1,1.0,0.333333,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,NaN,NaN
2,1.0,1.000000,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,NaN,NaN
3,1.0,0.333333,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,NaN,NaN
4,0.0,1.000000,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,NaN,NaN


In [15]:
test_meta = pd.DataFrame.copy(test_)
# test_meta["M1"] = np.nan
# test_meta["M2"] = np.nan
print(test_.shape)
test_meta.head()

(357, 8)


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0.333333,0.7750,0.000,0.000000,0.156150,0.0,0.0,0.0
1,1.000000,0.2750,0.000,0.000000,0.014680,1.0,0.0,1.0
2,0.666667,0.4125,0.125,0.333333,0.054164,0.0,0.0,1.0
3,1.000000,0.3500,0.000,0.000000,0.015412,1.0,0.0,1.0
4,1.000000,0.4000,0.000,0.000000,0.015127,1.0,1.0,0.0


### Part 2

For each train fold: combine the other 4 into a training set and predict on each fold i 
- train fold 1 = 2,3,4,5 (Predict on fold 1)
- train fold 2 = 1,3,4,5 (Predict on Fold 2)
- train fold 3 = 1,2,4,5 (Predict on Fold 3)
- train fold 4 = 1,2,3,5 (Predict on Fold 4)
- train fold 5 = 1,2,3,4 (Predict on Fold 5)

In [16]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = False)
for train_index, test_index in kf.split(train_):
    print("Train:", train_index, "Test:", test_index, "\n")

Train: [179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376
 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394
 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412
 413 414 415 416 417 418 419 420 421 422 423

In [17]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 0


In [18]:
"""Copy and Paste Output: Change Target Index In Folds If Necessary"""
j = 1
for train_index, test_index in kf.split(train):
    # Set Train Folds With All Data Rows
    print ('train_fold{} = train'.format(j))  
    # Seperate Test Fold Without Class
    print ('test_fold{} = train.ix[{}:{}, 1:]'.format(j, min(test_index), max(test_index) + 1))
    # Seperate Target Fold Using Only The Class
    print ('target_fold{} = train_fold{}.ix[:,0]'.format(j,j))
    # Seperate Features Out Of Test 
    print ('train_fold{} = train_fold{}.ix[{}:,1:]'.format(j,j, 0))
    # Drop Excluded Training Set
    print ('train_fold{} = train_fold{}.drop(train_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1))
    # Drop Excluded Target Set
    print ('target_fold{} = target_fold{}.drop(target_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1), "\n")
    j += 1

train_fold1 = train
test_fold1 = train.ix[0:179, 1:]
target_fold1 = train_fold1.ix[:,0]
train_fold1 = train_fold1.ix[0:,1:]
train_fold1 = train_fold1.drop(train_fold1.index[0:179])
target_fold1 = target_fold1.drop(target_fold1.index[0:179]) 

train_fold2 = train
test_fold2 = train.ix[179:357, 1:]
target_fold2 = train_fold2.ix[:,0]
train_fold2 = train_fold2.ix[0:,1:]
train_fold2 = train_fold2.drop(train_fold2.index[179:357])
target_fold2 = target_fold2.drop(target_fold2.index[179:357]) 

train_fold3 = train
test_fold3 = train.ix[357:535, 1:]
target_fold3 = train_fold3.ix[:,0]
train_fold3 = train_fold3.ix[0:,1:]
train_fold3 = train_fold3.drop(train_fold3.index[357:535])
target_fold3 = target_fold3.drop(target_fold3.index[357:535]) 

train_fold4 = train
test_fold4 = train.ix[535:713, 1:]
target_fold4 = train_fold4.ix[:,0]
train_fold4 = train_fold4.ix[0:,1:]
train_fold4 = train_fold4.drop(train_fold4.index[535:713])
target_fold4 = target_fold4.drop(target_fold4.index[535:713]) 

train_fold

In [19]:
train_fold1 = train
test_fold1 = train.ix[0:179, 1:]
target_fold1 = train_fold1.ix[:,0]
train_fold1 = train_fold1.ix[0:,1:]
train_fold1 = train_fold1.drop(train_fold1.index[0:179])
target_fold1 = target_fold1.drop(target_fold1.index[0:179]) 

train_fold2 = train
test_fold2 = train.ix[179:357, 1:]
target_fold2 = train_fold2.ix[:,0]
train_fold2 = train_fold2.ix[0:,1:]
train_fold2 = train_fold2.drop(train_fold2.index[179:357])
target_fold2 = target_fold2.drop(target_fold2.index[179:357]) 

train_fold3 = train
test_fold3 = train.ix[357:535, 1:]
target_fold3 = train_fold3.ix[:,0]
train_fold3 = train_fold3.ix[0:,1:]
train_fold3 = train_fold3.drop(train_fold3.index[357:535])
target_fold3 = target_fold3.drop(target_fold3.index[357:535]) 

train_fold4 = train
test_fold4 = train.ix[535:713, 1:]
target_fold4 = train_fold4.ix[:,0]
train_fold4 = train_fold4.ix[0:,1:]
train_fold4 = train_fold4.drop(train_fold4.index[535:713])
target_fold4 = target_fold4.drop(target_fold4.index[535:713]) 

train_fold5 = train
test_fold5 = train.ix[713:891, 1:]
target_fold5 = train_fold5.ix[:,0]
train_fold5 = train_fold5.ix[0:,1:]
train_fold5 = train_fold5.drop(train_fold5.index[713:891])
target_fold5 = target_fold5.drop(target_fold5.index[713:891]) 


In [20]:
print(train_fold1.shape)
print(target_fold1.shape)
print(test_fold1.shape) 

print(train_fold2.shape)
print(target_fold2.shape)
print(test_fold2.shape) 

print(train_fold3.shape)
print(target_fold3.shape)
print(test_fold3.shape) 

print(train_fold4.shape)
print(target_fold4.shape)
print(test_fold4.shape) 

print(train_fold5.shape)
print(target_fold5.shape)
print(test_fold5.shape) 

(712, 8)
(712,)
(180, 8)
(713, 8)
(713,)
(179, 8)
(713, 8)
(713,)
(179, 8)
(713, 8)
(713,)
(179, 8)
(713, 8)
(713,)
(178, 8)


### Part 3
- Use Model M1 to train on each train fold i and predict the excluded fold and store the values back into M1 for its fold ID
- Do the same with M2

In [21]:
"""Instantiate Models"""
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# m2 = RandomForestClassifier()
m2 = KNeighborsClassifier(n_neighbors = 25)
m1 = GradientBoostingClassifier(n_estimators = 100)
# m1 = LogisticRegression()

In [22]:
"""Train Data On First Model"""
train_folds = [train_fold1, train_fold2, train_fold3, train_fold4, train_fold5]
target_folds = [target_fold1, target_fold2, target_fold3, target_fold4, target_fold5]
test_folds = [test_fold1, test_fold2, test_fold3, test_fold4, test_fold5]

y_pred1 = []
for i, j in zip(train_folds, target_folds):
    m1.fit(i, j)
for k in test_folds:
    target_pred = m1.predict(k)
    y_pred1.append(target_pred) 
    
y_pred1 = np.concatenate(y_pred1, axis=0)
y_pred1 = pd.DataFrame(y_pred1)
train_meta["M1"] = y_pred1

In [23]:
"""Train Data On Second Model"""
y_pred2 = []
for i, j in zip(train_folds, target_folds):
    m2.fit(i, j)
for k in test_folds:
    target_pred2 = m2.predict(k)
    y_pred2.append(target_pred2) 
    
y_pred2 = np.concatenate(y_pred2, axis=0)
y_pred2 = pd.DataFrame(y_pred2)
train_meta["M2"] = y_pred2

### Part 4

Fit m1 Model to train_meta
- Put new predictions into train_meta m1 column
Fit m2 Model to train_meta
- Put new predictions into train_meta m2 column

In [24]:
train_meta

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,0.0,1.000000,0.2750,0.125,0.000000,0.014151,1.0,0.0,1.0,0.0,0.0
1,1.0,0.333333,0.4750,0.125,0.000000,0.139136,0.0,0.0,0.0,1.0,1.0
2,1.0,1.000000,0.3250,0.000,0.000000,0.015469,0.0,0.0,1.0,1.0,0.0
3,1.0,0.333333,0.4375,0.125,0.000000,0.103644,0.0,0.0,1.0,1.0,1.0
4,0.0,1.000000,0.4375,0.000,0.000000,0.015713,1.0,0.0,1.0,0.0,0.0
5,0.0,1.000000,0.0000,0.000,0.000000,0.016510,1.0,1.0,0.0,0.0,0.0
6,0.0,0.333333,0.6750,0.000,0.000000,0.101229,1.0,0.0,1.0,0.0,0.0
7,0.0,1.000000,0.0250,0.375,0.166667,0.041136,1.0,0.0,1.0,0.0,0.0
8,1.0,1.000000,0.3375,0.000,0.333333,0.021731,0.0,0.0,1.0,0.0,1.0
9,1.0,0.666667,0.1750,0.125,0.000000,0.058694,0.0,0.0,0.0,1.0,1.0


### Part 5
- Predict m1 on test_meta
- Predict m2 on test_meta

Part 4
Use another model (S) and use train_meta to make final predictions on test_meta 

In [25]:
train_meta.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,0.0,1.000000,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,0.0
1,1.0,0.333333,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,1.0,1.0
2,1.0,1.000000,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,1.0,0.0
3,1.0,0.333333,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,1.0,1.0
4,0.0,1.000000,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,0.0


### Store New Targets Elsewhere For Training And Delete Them Off Training Set

In [26]:
train_meta_Class = train_meta["Survived"]
train_meta_M1 = train_meta["M1"]
train_meta_M2 = train_meta["M2"]

del train_meta["Survived"]

training = pd.DataFrame.copy(train_meta)

del train_meta["M1"], train_meta["M2"]

In [27]:
train_meta.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.000000,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0
1,0.333333,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0
2,1.000000,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0
3,0.333333,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0
4,1.000000,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0


In [28]:
"""Train Data On Third Model"""
from sklearn.linear_model import LogisticRegression
m1.fit(train_meta, train_meta_M1)
target_pred_M1 = m1.predict(test_meta)

m2.fit(train_meta, train_meta_M2)
target_pred_M2 = m2.predict(test_meta)

test_meta["M1"] = target_pred_M1
test_meta["M2"] = target_pred_M2

In [29]:
training.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,1.000000,0.2750,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,0.0
1,0.333333,0.4750,0.125,0.0,0.139136,0.0,0.0,0.0,1.0,1.0
2,1.000000,0.3250,0.000,0.0,0.015469,0.0,0.0,1.0,1.0,0.0
3,0.333333,0.4375,0.125,0.0,0.103644,0.0,0.0,1.0,1.0,1.0
4,1.000000,0.4375,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,0.0


In [30]:
test_meta.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,0.333333,0.7750,0.000,0.000000,0.156150,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.2750,0.000,0.000000,0.014680,1.0,0.0,1.0,0.0,0.0
2,0.666667,0.4125,0.125,0.333333,0.054164,0.0,0.0,1.0,0.0,0.0
3,1.000000,0.3500,0.000,0.000000,0.015412,1.0,0.0,1.0,0.0,0.0
4,1.000000,0.4000,0.000,0.000000,0.015127,1.0,1.0,0.0,0.0,0.0


### Part 6 Predict On Actual Testing Set

In [31]:
s = LogisticRegression()
s.fit(training, train_meta_Class)
target_pred_Class = s.predict(test_meta)

test_meta["Survived"] = target_pred_Class

In [32]:
print(round(accuracy(targ_test, target_pred_Class)*100, 2), "% Accuracy")

80.39 % Accuracy


In [33]:
test_meta

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,M1,M2,Survived
0,0.333333,0.77500,0.000,0.000000,0.156150,0.0,0.0,0.0,0.0,0.0,1.0
1,1.000000,0.27500,0.000,0.000000,0.014680,1.0,0.0,1.0,0.0,0.0,0.0
2,0.666667,0.41250,0.125,0.333333,0.054164,0.0,0.0,1.0,0.0,0.0,1.0
3,1.000000,0.35000,0.000,0.000000,0.015412,1.0,0.0,1.0,0.0,0.0,0.0
4,1.000000,0.40000,0.000,0.000000,0.015127,1.0,1.0,0.0,0.0,0.0,0.0
5,1.000000,0.00000,0.000,0.000000,0.015412,1.0,0.0,1.0,0.0,0.0,0.0
6,0.333333,0.45000,0.125,0.333333,0.234224,0.0,0.0,1.0,1.0,0.0,1.0
7,1.000000,0.23750,0.000,0.000000,0.013175,1.0,1.0,0.0,0.0,0.0,0.0
8,1.000000,0.56250,0.000,0.000000,0.013614,1.0,0.0,1.0,0.0,0.0,0.0
9,1.000000,0.35000,0.000,0.000000,0.018543,1.0,0.0,1.0,0.0,0.0,0.0
